#### Thư viện và Khai báo hàm

In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

#### Nhu cầu hôm nay

Đây là phần chạy dưới local

In [2]:
#region Part 1
date_from_today = pd.to_datetime("today").strftime("%Y%m%d")
file_path = 'C:/Users/minhd/Bách hóa XANH/DATA/RAW-DATA/report-911/DATA-911-FRESH-' + str(date_from_today) + '.xlsx'
df_911 = pd.read_excel(file_path, usecols=['Ngày nhận hàng','Tên sản phẩm','Tên siêu thị','Số lượng cần mua'])
#Thêm các cột mã
df_911['Mã sản phẩm'] = df_911['Tên sản phẩm'].str.extract('(^[0-9]{0,13})').astype('int64')
df_911['Mã siêu thị'] = df_911['Tên siêu thị'].str.extract('(^[0-9]{0,5})').astype('int64')

df_996 = pd.read_excel(r'D:\PROJECT 101\tools\Kiểm tra nhu cầu siêu thị khai trương\Khai báo\list_product.xlsx')
df_cudai = pd.read_excel(r'D:\PROJECT 101\tools\Kiểm tra nhu cầu siêu thị khai trương\Khai báo\list_củ quả dài ngày.xlsx')
df_cudai['Nhóm hàng 2'] = 'Rau địa phương - Củ dài ngày'
df_996 = pd.merge(df_996, df_cudai[['Mã sản phẩm','Nhóm hàng 2']], how = 'left', on = 'Mã sản phẩm')

df_911 = pd.merge(df_911, df_996[['Mã sản phẩm','Ngành hàng','Nhóm hàng','Nhóm hàng 2','Ngành hàng - Phân tích']],
                  how = 'left', on = 'Mã sản phẩm')
# Chỉ fillna cột 'Nhóm hàng 2' khi 'Ngành hàng - Phân tích' là 'Rau đà lạt'
df_911.loc[df_911['Ngành hàng - Phân tích'] == 'Rau Đà Lạt', 'Nhóm hàng 2'] = \
    df_911.loc[df_911['Ngành hàng - Phân tích'] == 'Rau Đà Lạt', 'Nhóm hàng 2'].fillna(df_911['Ngành hàng - Phân tích'])
df_911['Nhóm hàng 2'] = df_911['Nhóm hàng 2'].fillna(df_911['Nhóm hàng'])
#endregion

#region Part 2
df_911_copy = df_911.copy()
df_nhomhang2 = df_911[['Mã sản phẩm','Nhóm hàng 2']].drop_duplicates()
df_nhomhang2['Mã sản phẩm'] = df_nhomhang2['Mã sản phẩm'].apply(lambda x: '{:.0f}'.format(x)).astype(str)

file_path = 'C:/Users/minhd/Bách hóa XANH/DATA/RAW-DATA/report-135/DATA-135-FRESH-' + str(date_from_today) + '.xlsx'
df_135 = pd.read_excel(file_path, usecols=['Thời gian gửi','Mã siêu thị','Mã sản phẩm','Tên sản phẩm'])

df_135['Mã sản phẩm'] = df_135['Mã sản phẩm'].apply(lambda x: '{:.0f}'.format(x)).astype(str)

df_135 = pd.merge(df_135, df_nhomhang2[['Mã sản phẩm','Nhóm hàng 2']], how = 'left', on = 'Mã sản phẩm')
df_135['Notify'] = 1

df_911_copy['Mã sản phẩm'] = df_911_copy['Mã sản phẩm'].apply(lambda x: '{:.0f}'.format(x)).astype(str)
df_gop_911_135 = pd.merge(df_911_copy, df_135[['Mã siêu thị','Mã sản phẩm','Notify']], how = 'left', on = ['Mã siêu thị','Mã sản phẩm'])
#endregion

df_gop_911_135.to_parquet(r'D:\PROJECT 101\tools\Kiểm tra nhu cầu siêu thị khai trương\data.parquet', index = False)

ValueError: Usecols do not match columns, columns expected but not found: ['Thời gian gửi', 'Tên sản phẩm', 'Mã siêu thị', 'Mã sản phẩm'] (sheet: 0)

Đây là phần chạy trên nền web

In [ ]:
df = pd.read_parquet(r'D:\PROJECT 101\tools\Kiểm tra nhu cầu siêu thị khai trương\data.parquet')
df_list_opening_store = pd.read_excel(r'D:\PROJECT 101\tools\Kiểm tra nhu cầu siêu thị khai trương\Khai báo\list_opening_store.xlsx')
df = pd.merge(df, df_list_opening_store, how = 'left', on = 'Mã siêu thị')
df = df[~df['Ngày khai trương'].isna()]
df_pivot = df.pivot_table(index=['Ngày khai trương','Ngày nhận hàng','Mã siêu thị','Tên siêu thị','Ngành hàng','Nhóm hàng 2'],
                               values=['Mã sản phẩm','Số lượng cần mua','Notify'],
                               aggfunc={'Mã sản phẩm': 'count', 'Số lượng cần mua': 'sum', 'Notify':'sum'}).reset_index()
df_pivot['Số lượng cần mua'] = round(df_pivot['Số lượng cần mua'], 1)
df_pivot.sort_values(by=['Ngày khai trương','Mã siêu thị','Ngành hàng','Mã sản phẩm','Nhóm hàng 2','Ngày nhận hàng'],
                     ascending=[True,True,True,False,True,True], inplace=True)
df_pivot.rename(columns={'Mã sản phẩm':'Số lượng SKU', 'Notify':'Số SKU Notify', 'Số lượng cần mua':'Tổng nhu cầu', 'Notify':'Tổng SKU Notify'}, inplace=True)

df_pivot = df_pivot.sort_values(by=['Ngày khai trương','Mã siêu thị','Ngành hàng','Nhóm hàng 2','Ngày nhận hàng'],
                                ascending=[True,True,True,True,True])
df_dsst = pd.read_excel(r'D:\PROJECT 101\tools\Kiểm tra nhu cầu siêu thị khai trương\Khai báo\Danh sách siêu thị.xlsx', usecols=['Mã siêu thị','Miền'])
df_pivot = pd.merge(df_pivot, df_dsst[['Mã siêu thị','Miền']], how = 'left', on = 'Mã siêu thị')
df_pivot['Miền'] = df_pivot['Miền'].fillna('Chưa cập nhật')
df_pivot['Thứ (Khai trương)'] = pd.to_datetime(df_pivot['Ngày khai trương']).dt.day_name()
df_pivot = df_pivot[['Ngày khai trương','Thứ (Khai trương)','Ngày nhận hàng','Mã siêu thị','Tên siêu thị','Miền','Ngành hàng','Nhóm hàng 2','Số lượng SKU','Tổng nhu cầu','Tổng SKU Notify']]
df_pivot = df_pivot.reset_index(drop=True)

In [ ]:
df_pivot.head()

,Ngày khai trương,Thứ (Khai trương),Ngày nhận hàng,Mã siêu thị,Tên siêu thị,Miền,Ngành hàng,Nhóm hàng 2,Số lượng SKU,Tổng nhu cầu,Tổng SKU Notify
0,2025-04-12,Saturday,2025-04-26,14819,14819-BHX_CTH_TNO - Trung Kiên,Miền Tây,Rau Củ Các Loại,Củ quả CL,21,50.0,0.0
1,2025-04-12,Saturday,2025-04-25,14819,14819-BHX_CTH_TNO - Trung Kiên,Miền Tây,Rau Củ Các Loại,Hoa Tươi Các Loại,4,4.0,0.0
2,2025-04-12,Saturday,2025-04-25,14819,14819-BHX_CTH_TNO - Trung Kiên,Miền Tây,Rau Củ Các Loại,Nấm Các Loại,8,18.0,0.0
3,2025-04-12,Saturday,2025-04-26,14819,14819-BHX_CTH_TNO - Trung Kiên,Miền Tây,Rau Củ Các Loại,Nấm Các Loại,8,7.2,0.0
4,2025-04-12,Saturday,2025-04-26,14819,14819-BHX_CTH_TNO - Trung Kiên,Miền Tây,Rau Củ Các Loại,Rau CL,29,122.0,0.0
